In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [11]:
###change path below####
path = 'chi_example.csv' 

data = pd.read_csv(path)
data.set_index(['ab_test_name'],inplace=True)
data

,devices,opt_ins,not_opt_ins
ab_test_name,,,
Control,249135,224,248911
upsell,621864,1012,620852
upsell_discout_churn,621100,987,620113


In [12]:
data_stats = data[['opt_ins', 'not_opt_ins']].transpose()
data_stats

ab_test_name,Control,upsell,upsell_discout_churn
opt_ins,224,1012,987
not_opt_ins,248911,620852,620113


In [35]:

############################################################

####enter parameters below and do chi_square analysis#######

############################################################

df = data_stats
control_name = 'Control' # change to your experiment group name
test_name = 'upsell'# change to your experiment group name
metric = 'opt-ins rate' #change to your metric name


############################################################
######chi_2 analysis(no need to change below)###############
############################################################

observed_1 = data_stats [[control_name ,test_name]] 
chi_square, p_value, degree_of_freedom, ob_df = stats.chi2_contingency(observed= observed_1)

if p_value < 0.05:
    sig = 'significant'
else:
    sig = 'not significant'

start = "\033[1m"
end = "\033[0;0m"


print 'chi square results below:'
print ''
print '############################################################'
print ''
print '     Result for this experiment is %s' % (start + sig.upper() + end)
print ''
print '1) the p_value for %s is: %.5f' % (metric, p_value)
print '2) chi_square for %s is %.2f' % (metric, chi_square)
print '3) degree of freedom for %s is %d'  % (metric, degree_of_freedom)
print ''
print '############################################################'


chi square results below:

############################################################

     Result for this experiment is SIGNIFICANT

1) the p_value for opt-ins rate is: 0.00000
2) chi_square for opt-ins rate is 66.06
3) degree of freedom for opt-ins rate is 1

############################################################


23.82

# additional notes: 


**  1) for the observed dataframe, it doesn't matter if you transpose or not. **

In [ ]:
# practice(transposing contingence table or not doesn't matter to the final chi2 test result, it is the same)

In [4]:
np.random.seed(10)

# Sample data randomly at fixed probabilities
voter_race = np.random.choice(a= ["asian","black","hispanic","other","white"],
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

# Sample data randomly at fixed probabilities
voter_party = np.random.choice(a= ["democrat","independent","republican"],
                              p = [0.4, 0.2, 0.4],
                              size=1000)

voters = pd.DataFrame({"race":voter_race, 
                       "party":voter_party})

voter_tab = pd.crosstab(voters.race, voters.party, margins = True)

voter_tab.columns = ["democrat","independent","republican","row_totals"]

voter_tab.index = ["asian","black","hispanic","other","white","col_totals"]

observed = voter_tab.ix[0:5,0:3]   # Get table without totals for later use
voter_tab

/Users/xduan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,democrat,independent,republican,row_totals
asian,21,7,32,60
black,65,25,64,154
hispanic,107,50,94,251
other,15,8,15,38
white,189,96,212,497
col_totals,397,186,417,1000


In [5]:
expected =  np.outer(voter_tab["row_totals"][0:5],
                     voter_tab.ix["col_totals"][0:3]) / 1000

expected = pd.DataFrame(expected)

expected.columns = ["democrat","independent","republican"]
expected.index = ["asian","black","hispanic","other","white"]

expected

,democrat,independent,republican
asian,23,11,25
black,61,28,64
hispanic,99,46,104
other,15,7,15
white,197,92,207


In [6]:
observed

,democrat,independent,republican
asian,21,7,32
black,65,25,64
hispanic,107,50,94
other,15,8,15
white,189,96,212


In [7]:
stats.chi2_contingency(observed= observed)

(7.1693212801620589,
 0.51847939294884204,
 8,
 array([[  23.82 ,   11.16 ,   25.02 ],
        [  61.138,   28.644,   64.218],
        [  99.647,   46.686,  104.667],
        [  15.086,    7.068,   15.846],
        [ 197.309,   92.442,  207.249]]))

In [9]:
observed2 = observed.transpose()
observed2

,asian,black,hispanic,other,white
democrat,21,65,107,15,189
independent,7,25,50,8,96
republican,32,64,94,15,212


In [10]:
stats.chi2_contingency(observed= observed2)

(7.1693212801620598,
 0.51847939294884204,
 8,
 array([[  23.82 ,   61.138,   99.647,   15.086,  197.309],
        [  11.16 ,   28.644,   46.686,    7.068,   92.442],
        [  25.02 ,   64.218,  104.667,   15.846,  207.249]]))

 ** 2). parametric (Z-test) and non-parametric (chi-squared) methods for proportion test **

Very good reading 
 - **Comparison of two proportions: parametric (Z-test) and non-parametric (chi-squared) methods **
   - if we assume the probability of common success is approximate 0.5, and the number of games is very high (under these assumption, a binomial distribution is approximate a gaussian distribution). Then we can use z-test to analyze two proportions. Otherwise, just use chi-squared test.
   - A chi-square test for equality of two proportions is exactly the same thing as a zz-test, if we have the two assumptions above.
   
   - https://www.r-bloggers.com/comparison-of-two-proportions-parametric-z-test-and-non-parametric-chi-squared-methods/
    

In [1]:
from scipy import stats
z_scores = -2.969695

# Survival function (also defined as 1 - cdf, but sf is sometimes more accurate).
p_values1 = stats.norm.sf(abs(z_scores)) #one-sided

p_values2 = stats.norm.cdf(z_scores, loc = 0, scale = 1)

# p_values1 = p_values 2
print p_values1, p_values2


0.00149047776003 0.00149047776003


#### p_value from chi-square test is very similar to 2-sided p-value using p_pool t-test. See 3 examples below.

> example 1

In [57]:
###change path below####
path = 'chi_example.csv' 

data = pd.read_csv(path)
data.set_index(['ab_test_name'],inplace=True)
data

,devices,opt_ins,not_opt_ins
ab_test_name,,,
Control,30000,5000,25000
upsell,32000,5300,24800
upsell_discout_churn,31000,6800,24200


In [78]:
# z.prop(30, 65, 74, 103)
# x1,x2,n1,n2
# for example p1 = 1.00* 30/74, p2 = 1.00*65/103
import pandas as pd
import numpy as np
from scipy import stats


p1 = 1.00* 5000/30000
p2 = 1.00* 5300/32000
N1 = 30000
N2 = 32000


def get_pvalue(p1, p2 ,N1, N2):
    diff = p2 - p1
    p_pool = 1.00* (N1*p1 + N2*p2)/(N1+N2)
    print ('p2 and p1 are: %.4f and %.4f' %(p2, p1))
    ## get standard error below
    SE = np.sqrt(p_pool* (1-p_pool) * (1.00/N1 + 1.00/N2))
    
    ## if it is two-side give 2*, if one-side, just stats.norm.cdf(lift, loc = 0, scale = scale_val )
    Z_score = 1.00* diff/SE
    print ('Z score is: %.3f' %Z_score)
    p_value = 2* stats.norm.sf(abs(Z_score)) #one-sided
#     p_value = 2 * stats.norm.cdf(diff, loc = 0, scale = SE )
    return p_value


print ('use p_pool method')
get_pvalue(p1,p2,N1,N2)

use p_pool method
p2 and p1 are: 0.1656 and 0.1667
Z score is: -0.348


0.7276492976564044

In [60]:
data_stats = data[['opt_ins', 'not_opt_ins']].transpose()
data_stats

ab_test_name,Control,upsell,upsell_discout_churn
opt_ins,5000,5300,6800
not_opt_ins,25000,24800,24200


In [74]:
observed_1 = data_stats[['Control', 'upsell']]
observed_1

ab_test_name,Control,upsell
opt_ins,5000,5300
not_opt_ins,25000,24800


In [79]:
p1 = 1.00* 5000/30000
p2 = 1.00* 5300/32000
N1 = 30000
N2 = 32000


observed_2 = observed_1
observed_2['Control'] = [N1*p1, N1*(1-p1)]
observed_2['upsell'] = [N2*p2, N2*(1-p2)]
observed_2 

/Users/xduan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/xduan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


ab_test_name,Control,upsell
opt_ins,5000.0,5300.0
not_opt_ins,25000.0,26700.0


In [81]:
observed_1

ab_test_name,Control,upsell
opt_ins,5000.0,5300.0
not_opt_ins,25000.0,26700.0


In [80]:
chi_square, p_value, degree_of_freedom, ob_df = stats.chi2_contingency(observed= observed_2)
print p_value, chi_square

0.735771438055 0.113878107923


> example 2. p_value of chi-square and t-test is different, but they lead to the same direction

In [ ]:
# z.prop(30, 65, 74, 103)
# x1,x2,n1,n2
# for example p1 = 1.00* 30/74, p2 = 1.00*65/103

In [53]:
observed_2 = observed_1
observed_2['Control'] = [30, 44]
observed_2['upsell'] = [65, 38]
observed_2



/Users/xduan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/xduan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


ab_test_name,Control,upsell
opt_ins,30,65
not_opt_ins,44,38


In [54]:

chi_square, p_value, degree_of_freedom, ob_df = stats.chi2_contingency(observed= observed_2)
print p_value, chi_square


0.0048490521073 7.93489200827


In [56]:
# z.prop(30, 65, 74, 103)
# x1,x2,n1,n2
# for example p1 = 1.00* 30/74, p2 = 1.00*65/103
import pandas as pd
import numpy as np
from scipy import stats


p1 = 1.00* 30/74
p2 = 1.00* 65/103
N1 = 74
N2 = 103



def get_pvalue(p1, p2 ,N1, N2):
    diff = p2 - p1
    p_pool = 1.00* (N1*p1 + N2*p2)/(N1+N2)
    print ('p2 and p1 are: %.4f and %.4f' %(p2, p1))
    ## get standard error below
    SE = np.sqrt(p_pool* (1-p_pool) * (1.00/N1 + 1.00/N2))
    
    ## if it is two-side give 2*, if one-side, just stats.norm.cdf(lift, loc = 0, scale = scale_val )
    Z_score = 1.00* diff/SE
    print ('Z score is: %.3f' %Z_score)
    p_value = 2* stats.norm.sf(abs(Z_score)) #two-sided
#     p_value = 2 * stats.norm.cdf(diff, loc = 0, scale = SE )
    print ('p_value is: %.6f' %p_value)


print ('use p_pool method')
get_pvalue(p1,p2,N1,N2)

use p_pool method
p2 and p1 are: 0.6311 and 0.4054
Z score is: 2.970
p_value is: 0.002981


> example 3

In [75]:
p1 = 0.01 * 0.68
p2 = 0.01 * 0.67
N1 = 340045
N2 = 338814



def get_pvalue(p1, p2 ,N1, N2):
    diff = p2 - p1
    p_pool = 1.00* (N1*p1 + N2*p2)/(N1+N2)
    print ('p2 and p1 are: %.4f and %.4f' %(p2, p1))
    ## get standard error below
    SE = np.sqrt(p_pool* (1-p_pool) * (1.00/N1 + 1.00/N2))
    
    ## if it is two-side give 2*, if one-side, just stats.norm.cdf(lift, loc = 0, scale = scale_val )
    Z_score = 1.00* diff/SE
    print ('Z score is: %.3f' %Z_score)
    p_value = 2* stats.norm.sf(abs(Z_score)) #two-sided
#     p_value = 2 * stats.norm.cdf(diff, loc = 0, scale = SE )
    print ('p_value is: %.6f' %p_value)


print ('use p_pool method')
get_pvalue(p1,p2,N1,N2)

use p_pool method
p2 and p1 are: 0.0067 and 0.0068
Z score is: -0.503
p_value is: 0.614877


In [76]:
p1 = 0.01 * 0.68
p2 = 0.01 * 0.67
N1 = 340045
N2 = 338814



observed_2 = observed_1
observed_2['Control'] = [N1*p1, N1*(1-p1)]
observed_2['upsell'] = [N2*p2, N2*(1-p2)]
observed_2 

/Users/xduan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/xduan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


ab_test_name,Control,upsell
opt_ins,2312.306,2270.0538
not_opt_ins,337732.694,336543.9462


In [77]:
chi_square, p_value, degree_of_freedom, ob_df = stats.chi2_contingency(observed= observed_2)
print p_value, chi_square


0.625336449558 0.23843814752


- this two-sided p_values equal to use the p-value using stats.chi2_contingency(observed= observed_1) in the R-blog example

In [6]:
p_values = stats.norm.sf(abs(z_scores))*2 #twosided
p_values

0.0029809555200650032